In [1]:
# Set CWD to correct directory
# Do not run more than once

import os
# wd = "C:/Users/lilba/Documents/COLLEGE/RIT Research/Interception_UXF_Analysis-CalibAssessment/Modules"
wd = os.getcwd()
os.chdir('/'.join(wd.split('/')[:-1])+'/')
print('CWD:'+os.getcwd())

CWD:/Users/gjdpci/Dropbox/Code/Catching - IPD/expansion analysis - UXF 1/Interception_UXF_Analysis


In [2]:
import sys

sys.path.append("Modules/")
sys.path.append("/")


import logging
import pickle
import numpy as np
import pandas as pd


fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

from loadData import unpackSession

# sys.path.append("pyFiles/")
# from processData import *

## Import raw subject data (slow) or previously imported data from file (fast)
**Set doNotLoad to False to check if pickle file exists and quickly load it instead of slowly generating new data.**

In [3]:
doNotLoad = False
subNum = 0
sessionDict = unpackSession(subNum, doNotLoad=doNotLoad)

INFO_loadData-unpackSession(): - Processing session: Data/P_201218121321_sub1
INFO_loadData-unpackSession(): - Importing session dict from pickle.


***> 0: P_201218121321_sub1
1: P_201218121321_sub1c
2: P_201218121321_sub1.zip
3: P_201218121321_sub1b
4: P_200917094202
5: P_201219105516


In [4]:
list(sessionDict.keys())

['subID',
 'trialInfo',
 'expConfig',
 'rawExpUnity',
 'rawExpGaze',
 'processedExp',
 'rawCalibUnity',
 'rawCalibGaze',
 'processedCalib',
 'analysisParameters']

In [5]:
sessionDict['trialInfo'].keys()

MultiIndex([(                               'ballFinalPos', 'x'),
            (                               'ballFinalPos', 'y'),
            (                               'ballFinalPos', 'z'),
            (                             'ballInitialPos', 'x'),
            (                             'ballInitialPos', 'y'),
            (                             'ballInitialPos', 'z'),
            (                             'ballInitialVel', 'x'),
            (                             'ballInitialVel', 'y'),
            (                             'ballInitialVel', 'z'),
            (                                'blockNumber',  ''),
            (                   'camera_movement_filename',  ''),
            (                         'contactLocOnPaddle', 'x'),
            (                         'contactLocOnPaddle', 'y'),
            (                         'contactLocOnPaddle', 'z'),
            (                          'contactLocinWorld', 'x'),
          

In [6]:
numTrials = len(sessionDict['trialInfo'])
trials = range(numTrials)
print('Total trials: ', numTrials)

Total trials:  86


In [7]:
aTrialsInfo = sessionDict['trialInfo'].loc[trials[0]]
print('Trial number: {tNum} \nTrial type: {tType}'.format(tNum = int(aTrialsInfo['trialNumber']),
                                                   tType = str(aTrialsInfo['trialType'].values)))

Trial number: 1 
Trial type: ['CalibrationAssessment']


In [8]:
gbProcessedCalib_trial = sessionDict['processedCalib'].groupby(['trialNumber'])
gbProcessedCalib_trial.groups.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86])

In [9]:
gbCurrGroup = gbProcessedCalib_trial.get_group(trials[0]+1)  # Add 1 to convert from zero-based to one-based index
gbCurrGroup.keys()

MultiIndex([(         'azimuthWidth',     ''),
            (            'base_data',     ''),
            (          'blockNumber',     ''),
            (               'camera', 'R0C0'),
            (               'camera', 'R0C1'),
            (               'camera', 'R0C2'),
            (               'camera', 'R0C3'),
            (               'camera', 'R1C0'),
            (               'camera', 'R1C1'),
            (               'camera', 'R1C2'),
            (               'camera', 'R1C3'),
            (               'camera', 'R2C0'),
            (               'camera', 'R2C1'),
            (               'camera', 'R2C2'),
            (               'camera', 'R2C3'),
            (               'camera', 'R3C0'),
            (               'camera', 'R3C1'),
            (               'camera', 'R3C2'),
            (               'camera', 'R3C3'),
            (            'cameraPos',    'x'),
            (            'cameraPos',    'y'),
            (

# Create graphs for azimuth and elevation, comparing against local target
* Separated by trial number
* Axis range is set at the top

# Calc cyclopean 

sessionDict['processedCalib']['cycEIH']

sessionDict['processedCalib']['cycEIH_el']

sessionDict['processedCalib']['cycEIH_az']

In [10]:
def calcCycEIH(sessionDictIn):
    
    gazeNormal2 = np.nanmean([sessionDictIn['processedCalib']['gaze-normal0'],sessionDictIn['processedCalib']['gaze-normal1']],axis=0)

    sessionDictIn['processedCalib'][('cycEIH','x')] = gazeNormal2[:,0]
    sessionDictIn['processedCalib'][('cycEIH','y')] = gazeNormal2[:,1]
    sessionDictIn['processedCalib'][('cycEIH','z')] = gazeNormal2[:,2]

    # Normalize!
    sessionDictIn['processedCalib']['cycEIH'] = np.array([np.divide(XYZ,np.linalg.norm(XYZ)) for XYZ in sessionDict['processedCalib']['cycEIH'].values],dtype=np.float)
    
    # While we're at it, let's calc spherical coordinates az / el
    sessionDictIn['processedCalib']['cycEIH_el'] = sessionDictIn['processedCalib'].apply(lambda row: np.rad2deg(np.arctan2(row[('cycEIH','y')],row[('cycEIH','z')])),axis=1)
    sessionDictIn['processedCalib']['cycEIH_az'] = sessionDictIn['processedCalib'].apply(lambda row: np.rad2deg(np.arctan2(row[('cycEIH','x')],row[('cycEIH','z')])),axis=1)
    
    logger.info('Added sessionDict[\'processedCalib\'][\'cycEIH\']')
    logger.info('Added sessionDict[\'processedCalib\'][\'cycEIH_az\']')
    logger.info('Added sessionDict[\'processedCalib\'][\'cycEIH_el\']')
    
    return sessionDictIn

sessionDict = calcCycEIH(sessionDict)


/Users/gjdpci/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/gjdpci/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
INFO___main__-calcCycEIH(): - Added sessionDict['processedCalib']['cycEIH']
INFO___main__-calcCycEIH(): - Added sessionDict['processedCalib']['cycEIH_az']
INFO___main__-calcCycEIH(): - Added sessionDict['processedCalib']['cycEIH_el']


# Filter data

In [11]:

def filterEIH(sessionDict):

    sgWinSizeSamples = sessionDict['analysisParameters']['sgWinSizeSamples']
    sgPolyorder = sessionDict['analysisParameters']['sgPolyorder']
    medFiltSize = sessionDict['analysisParameters']['medFiltSize']
    interpResS = sessionDict['analysisParameters']['interpResS']
    sgWinSizeSamples = sessionDict['analysisParameters']['sgWinSizeSamples']

    from scipy.signal import savgol_filter

    # FIlter
    proc = sessionDict['processedCalib']

    # get ∆T per frame
    frameDur = np.float(proc['frameTime'].diff().mode()[0])
    
    # Median filter to remove outliers
    proc['cycEIHFilt_az'] = proc['cycEIH_az'].rolling(medFiltSize).median()
    proc['cycEIHFilt_el'] = proc['cycEIH_el'].rolling(medFiltSize).median()

    # Fill in empty values / nans
    proc['cycEIHFilt_az'] = proc['cycEIH_az'].fillna(0)
    proc['cycEIHFilt_el'] = proc['cycEIH_el'].fillna(0)

    # Savitsky-Golay filter for smoothing
    proc['cycEIHFilt_az'] = savgol_filter(proc['cycEIHFilt_az'],
                                                        sgWinSizeSamples,
                                                        sgPolyorder,
                                                        deriv=0,
                                                        delta = frameDur,
                                                        axis=0,
                                                        mode='interp')

    proc['cycGIWFilt_el'] = savgol_filter(proc['cycEIHFilt_el'],
                                                        sgWinSizeSamples,
                                                        sgPolyorder,
                                                        deriv=0,
                                                        delta = frameDur,
                                                        axis=0,
                                                        mode='interp')

    sessionDict['processedCalib'] = proc
    
    logger.info('Added sessionDict[\'processedCalib\'][\'cycEIHFilt_az\']')
    logger.info('Added sessionDict[\'processedCalib\'][\'cycEIHFilt_el\']')

    return sessionDict

sessionDict = filterEIH(sessionDict)

INFO___main__-filterEIH(): - Added sessionDict['processedCalib']['cycEIHFilt_az']
INFO___main__-filterEIH(): - Added sessionDict['processedCalib']['cycEIHFilt_el']


# Compare raw and filtered signal for a trial

In [12]:
%matplotlib widget
# %matplotlib inline

In [55]:
import matplotlib.pyplot as plt


tNum = 84 # Trial to plot
yRange = 30 # +/- y degrees
xRange_minMax = [0,1] # [min, max] seconds

gbProcessedCalib_trial = sessionDict['processedCalib'].groupby(['trialNumber'])
trData = gbProcessedCalib_trial.get_group(tNum)

timestamps = trData['pupilTimestamp'] - trData['pupilTimestamp'].iloc[0]

p = plt.figure(figsize=(15, 10))
plt.style.use('ggplot')

grid = plt.GridSpec(2, 2)#, wspace=0.4, hspace=0.3)
ax = p.add_subplot(grid[:2,:2])
# ax2 = p.add_subplot(grid[:2,:2])

# ax.plot(timestamps,trData['cycEIH_az'],color='c',linewidth=3,alpha = 1.0,linestyle=':',label='raw az')

ax.scatter(timestamps,trData['cycEIH_az'],s=20,color='c',alpha = 1.0,linestyle=':',label='raw az')
ax.plot(timestamps,trData['cycEIHFilt_az'],linestyle='-',color='k',linewidth=2,alpha = 0.2,label='filt az')
ax.scatter(timestamps,trData['cycEIHFilt_az'],s=20,color='k',linewidth=2,alpha = 0.2)

ax.plot(timestamps,trData['cycEIH_el'],color='b',linewidth=3,alpha = 1.0,linestyle=':',label='raw el')
ax.plot(timestamps,trData['cycEIHFilt_el'],linestyle='-',color='g',linewidth=2,alpha = 0.2,label='filt az')
ax.scatter(timestamps,trData['cycEIHFilt_el'],s=20,color='g',linewidth=2,alpha = 0.2)

ax.set_xlabel('time (seconds)')
ax.set_xlim(-xRange_minMax[0], xRange_minMax[1])

ax.set_ylabel('degrees')
ax.set_ylim(-yRange, yRange)


ax.legend(loc= 'upper left')

# fig.figure.savefig('Output/'+str(trialRowIdx+1)+"_azimuth.png")
# plt.pyplot.close(fig.figure)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …